#### Given a dataset with raw attributes (e.g., sales transactions), create new features (e.g., total revenue, customer frequency, category-based sales). Apply discretization, transformation, and outlier detection to improve dataset quality.

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
import warnings

# Suppress warnings for cleaner output (e.g., for pd.to_datetime('now'))
warnings.filterwarnings('ignore')

In [2]:
#%%
# Create a raw dataset of sales transactions
data = {
    'invoice_id': ['A1001', 'A1001', 'A1002', 'A1003', 'A1003', 'A1003', 
                   'A1004', 'A1005', 'A1005', 'A1006', 'A1007', 'A1008'],
    'customer_id': [101, 101, 102, 101, 101, 101, 
                    103, 102, 102, 101, 104, 104],
    'product_category': ['Electronics', 'Clothing', 'Electronics', 'Books', 'Clothing', 'Home', 
                         'Beauty', 'Electronics', 'Home', 'Books', 'Electronics', 'Electronics'],
    'quantity': [1, 2, 1, 3, 1, 1, 
                 2, 1, 5, 1, 1, 1],
    'unit_price': [120, 30, 150, 15, 25, 50, 
                   20, 130, 40, 18, 5000, 4800], # Added large values for outlier detection
    'invoice_date': [
        '2023-01-10', '2023-01-10', '2023-01-11', '2023-01-15', '2023-01-15', '2023-01-15',
        '2023-01-20', '2023-01-22', '2023-01-22', '2023-02-01', '2023-02-05', '2023-02-06'
    ]
}

df = pd.DataFrame(data)
df['invoice_date'] = pd.to_datetime(df['invoice_date'])

print("--- Raw Transactional Data ---")
print(df)
df.info()

--- Raw Transactional Data ---
   invoice_id  customer_id product_category  quantity  unit_price invoice_date
0       A1001          101      Electronics         1         120   2023-01-10
1       A1001          101         Clothing         2          30   2023-01-10
2       A1002          102      Electronics         1         150   2023-01-11
3       A1003          101            Books         3          15   2023-01-15
4       A1003          101         Clothing         1          25   2023-01-15
5       A1003          101             Home         1          50   2023-01-15
6       A1004          103           Beauty         2          20   2023-01-20
7       A1005          102      Electronics         1         130   2023-01-22
8       A1005          102             Home         5          40   2023-01-22
9       A1006          101            Books         1          18   2023-02-01
10      A1007          104      Electronics         1        5000   2023-02-05
11      A1008        

In [3]:
#%%
# Create a total_price column (quantity * unit_price)
df['total_price'] = df['quantity'] * df['unit_price']

print("--- Data with total_price ---")
print(df[['invoice_id', 'product_category', 'quantity', 'unit_price', 'total_price']])

--- Data with total_price ---
   invoice_id product_category  quantity  unit_price  total_price
0       A1001      Electronics         1         120          120
1       A1001         Clothing         2          30           60
2       A1002      Electronics         1         150          150
3       A1003            Books         3          15           45
4       A1003         Clothing         1          25           25
5       A1003             Home         1          50           50
6       A1004           Beauty         2          20           40
7       A1005      Electronics         1         130          130
8       A1005             Home         5          40          200
9       A1006            Books         1          18           18
10      A1007      Electronics         1        5000         5000
11      A1008      Electronics         1        4800         4800


In [4]:
#%%
# Define a "snapshot date" to calculate recency (e.g., the day after the last transaction)
snapshot_date = df['invoice_date'].max() + pd.Timedelta(days=1)
print(f"Snapshot date for recency: {snapshot_date.date()}")

# Aggregate data at the customer level
customer_df = df.groupby('customer_id').agg(
    total_revenue=('total_price', 'sum'),
    customer_frequency=('invoice_id', 'nunique'),
    most_recent_purchase=('invoice_date', 'max')
)

# Calculate recency (days_since_last_purchase)
customer_df['days_since_last_purchase'] = (snapshot_date - customer_df['most_recent_purchase']).dt.days

# Drop the original date column as it's no longer needed
customer_df = customer_df.drop(columns=['most_recent_purchase'])

print("\n--- Aggregated Customer Data ---")
print(customer_df)

Snapshot date for recency: 2023-02-07

--- Aggregated Customer Data ---
             total_revenue  customer_frequency  days_since_last_purchase
customer_id                                                             
101                    318                   3                         6
102                    480                   2                        16
103                     40                   1                        18
104                   9800                   2                         1


In [5]:
#%%
# Create category-based sales features using pivot_table
category_sales = df.pivot_table(
    index='customer_id',
    columns='product_category',
    values='total_price',
    aggfunc='sum'
).fillna(0) # Fill NaNs with 0 for customers who didn't buy in a category

# Add a prefix to the column names for clarity
category_sales = category_sales.add_prefix('sales_')

print("--- Customer Sales by Category ---")
print(category_sales)

# Merge this back into our main customer_df
customer_df = customer_df.join(category_sales)

print("\n--- Combined Customer Data ---")
print(customer_df)

--- Customer Sales by Category ---
product_category  sales_Beauty  sales_Books  sales_Clothing  \
customer_id                                                   
101                        0.0         63.0            85.0   
102                        0.0          0.0             0.0   
103                       40.0          0.0             0.0   
104                        0.0          0.0             0.0   

product_category  sales_Electronics  sales_Home  
customer_id                                      
101                           120.0        50.0  
102                           280.0       200.0  
103                             0.0         0.0  
104                          9800.0         0.0  

--- Combined Customer Data ---
             total_revenue  customer_frequency  days_since_last_purchase  \
customer_id                                                                
101                    318                   3                         6   
102                    480

In [6]:
#%%
# Discretize 'customer_frequency' into groups
# We'll define bins: 1 = Low, 2 = Medium, 3+ = High
bins = [0, 1, 2, np.inf]
labels = ['Low', 'Medium', 'High']

customer_df['frequency_group'] = pd.cut(
    customer_df['customer_frequency'], 
    bins=bins, 
    labels=labels, 
    right=True
)

print("--- Data with Discretized Frequency ---")
print(customer_df[['customer_frequency', 'frequency_group']])

--- Data with Discretized Frequency ---
             customer_frequency frequency_group
customer_id                                    
101                           3            High
102                           2          Medium
103                           1             Low
104                           2          Medium


In [7]:
#%%
# 1. Log Transformation for skewed data (e.g., revenue)
# We use np.log1p which calculates log(1 + x) to handle potential zero values
customer_df['log_revenue'] = np.log1p(customer_df['total_revenue'])

print("\n--- Log-Transformed Revenue ---")
print(customer_df[['total_revenue', 'log_revenue']])

# 2. Standardization (Z-score scaling)
scaler = StandardScaler()

# We fit_transform the scaler. Note the .values and reshaping.
scaled_revenue = scaler.fit_transform(customer_df[['total_revenue']].values)
customer_df['scaled_revenue'] = scaled_revenue

print("\n--- Standard-Scaled Revenue ---")
print(customer_df[['total_revenue', 'scaled_revenue']])


--- Log-Transformed Revenue ---
             total_revenue  log_revenue
customer_id                            
101                    318     5.765191
102                    480     6.175867
103                     40     3.713572
104                   9800     9.190240

--- Standard-Scaled Revenue ---
             total_revenue  scaled_revenue
customer_id                               
101                    318       -0.567558
102                    480       -0.528290
103                     40       -0.634942
104                   9800        1.730790


In [8]:
#%%
# Outlier Detection for 'total_revenue' using the IQR method
feature = 'total_revenue'
Q1 = customer_df[feature].quantile(0.25)
Q3 = customer_df[feature].quantile(0.75)
IQR = Q3 - Q1

lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

print(f"--- Outlier Detection for '{feature}' ---")
print(f"Q1: {Q1:.2f}")
print(f"Q3: {Q3:.2f}")
print(f"IQR: {IQR:.2f}")
print(f"Lower Bound (Outliers <): {lower_bound:.2f}")
print(f"Upper Bound (Outliers >): {upper_bound:.2f}")

# Identify and flag the outliers
customer_df['is_revenue_outlier'] = (customer_df[feature] < lower_bound) | (customer_df[feature] > upper_bound)

print("\n--- Customers Flagged as Outliers ---")
print(customer_df[customer_df['is_revenue_outlier']])

--- Outlier Detection for 'total_revenue' ---
Q1: 248.50
Q3: 2810.00
IQR: 2561.50
Lower Bound (Outliers <): -3593.75
Upper Bound (Outliers >): 6652.25

--- Customers Flagged as Outliers ---
             total_revenue  customer_frequency  days_since_last_purchase  \
customer_id                                                                
104                   9800                   2                         1   

             sales_Beauty  sales_Books  sales_Clothing  sales_Electronics  \
customer_id                                                                 
104                   0.0          0.0             0.0             9800.0   

             sales_Home frequency_group  log_revenue  scaled_revenue  \
customer_id                                                            
104                 0.0          Medium      9.19024         1.73079   

             is_revenue_outlier  
customer_id                      
104                        True  


In [9]:
#%%
print("\n--- FINAL PROCESSED CUSTOMER DATASET ---")

# Re-ordering columns for a cleaner view
all_columns = [
    # Original Aggregated
    'total_revenue', 
    'customer_frequency', 
    'days_since_last_purchase',
    # Category Sales
    'sales_Beauty', 
    'sales_Books', 
    'sales_Clothing', 
    'sales_Electronics', 
    'sales_Home',
    # Discretized
    'frequency_group',
    # Transformed
    'log_revenue', 
    'scaled_revenue',
    # Outlier Flag
    'is_revenue_outlier'
]

# Ensure all columns exist, in case a category wasn't in the sample data
final_cols = [col for col in all_columns if col in customer_df.columns]
print(customer_df[final_cols].to_markdown(floatfmt=".2f"))

print("\n\nFinal Data Info:")
customer_df.info()


--- FINAL PROCESSED CUSTOMER DATASET ---
|   customer_id |   total_revenue |   customer_frequency |   days_since_last_purchase |   sales_Beauty |   sales_Books |   sales_Clothing |   sales_Electronics |   sales_Home | frequency_group   |   log_revenue |   scaled_revenue | is_revenue_outlier   |
|--------------:|----------------:|---------------------:|---------------------------:|---------------:|--------------:|-----------------:|--------------------:|-------------:|:------------------|--------------:|-----------------:|:---------------------|
|           101 |             318 |                    3 |                          6 |           0.00 |         63.00 |            85.00 |              120.00 |        50.00 | High              |          5.77 |            -0.57 | False                |
|           102 |             480 |                    2 |                         16 |           0.00 |          0.00 |             0.00 |              280.00 |       200.00 | Medium          